## Dramatron

[Dramatron](https://arxiv.org/pdf/2209.14958.pdf) is a paper published by Deepmind in 2022 describing an approach for procedurally generating stories. It starts with a logline provided by the user (e.g. "A farm boy from a desert planet joins a rebellion against an evil empire") and uses that to first generate a title and characters, which are then used to generate subsequent content like locations and scene beats. Figure 1. from the paper outlines this approach in full. 


<img src="./dramatron.png" />

In this post, we'll walk through the implementation of Dramatron in Python using `promptx` - a library to help interact with generative models and embeddings.

```bash
#install promptx
pip install pxx
```

`promptx` comes with a cli which we'll use to create a new project. This is used to configure the models and define how the embedding data is stored. 

```bash
px init dramatron
```

Now we have a project setup we can access it using the `1oad` function in any notebook or script in the same project directory. 

`promptx` lets you define the expected output of a prompt by passing in a Pydantic schema. This is used to both provide guidance for what should be generated and validation that the correct data was created.

Let's create the basic types we'll need to implement Dramatron.

In [1]:
from typing import List
from pydantic import BaseModel, Field


class Character(BaseModel):
    name: str
    description: str


class Location(BaseModel):
    name: str
    description: str


class SceneBeat(BaseModel):
    location: str
    plot_element: str
    description: str


class Story(BaseModel):
    logline: str
    title: str = None
    outline: List[SceneBeat] = None
    characters: List[Character] = None
    locations: List[Location] = None

    def __init__(self, logline, **kwargs):
        super().__init__(logline=logline, **kwargs)

Next, lets define some examples, which will be used as few shot guidance for the model. I'm using the examples from the paper for consistency. 

In [2]:
star_wars = Story(
    title="Star Wars",
    logline='''
    A science - fiction fantasy about a naive but ambitious farm boy from a 
    backwater desert who discovers powers he never knew he had when he teams 
    up with a feisty princess, a mercenary space pilot and an old wizard warrior 
    to lead a ragtag rebellion against the sinister forces of the evil Galactic 
    Empire.
    ''',
    characters=[
        Character(
            name='Luke Skywalker',
            description='''
            Luke Skywalker is the hero. A naive farm boy, he will 
            discover special powers under the guidance of mentor 
            Ben Kenobi.
            ''',
        ),
        Character(
            name='Ben Kenobi',
            description='''
            Ben Kenobi is the mentor figure. A recluse Jedi warrior, 
            he will take Luke Skywalker as apprentice .
            ''',
        ),
        Character(
            name='Dartha Vader',
            description='''
            Darth Vader is the antagonist. As a commander of the 
            evil Galactic Empire, he controls space station The 
            Death Star.
            ''',
        ),
        Character(
            name='Princess Leia',
            description='''
            Princess Leia holds the plans of the Death Star. She is 
            feisty and brave. She will become Luke's friend.
            ''',
        ),
        Character(
            name='Han Solo',
            description='''
            Han Solo is a brash mercenary space pilot of the 
            Millenium Falcon and a friend of Chebacca. He will
            take Luke on his spaceship.
            ''',
        ),
        Character(
            name='Chewbacca',
            description='''
            Chewbacca is a furry and trustful monster. He is a friend 
            of Han Solo and a copilot on the Millemium Falcon.
            ''',
        ),
    ],
    locations=[
        Location(
            name='Farm',
            description='The farm is a desert planet where Luke Skywalker lives',
        ),
    ],
    outline=[
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element='The Ordinary World',
            description='Luke Skywalker is living a normal and humble life as a farm boy on his home planet.',
        ),
        SceneBeat(
            location='Desert of Tatooine',
            plot_element='Call to Adventure',
            description='''
            Luke is called to his adventure by robot R2-D2 and Ben Kenobi. 
            Luke triggers R2-D2's message from Princess Leia and is intrigued 
            by her message. When R2-D2 escapes to find Ben Kenobi, Luke follows 
            and is later saved by Kenobi, who goes on to tell Luke about his Jedi 
            heritage. Kenobi suggests that he should come with him.
            '''
        ),
        SceneBeat(
            location="Ben Kenobi's farm",
            plot_element='Refusal of the Call',
            description='''
            Luke refuses Kenobi, telling him that he can take Kenobi and the 
            droids as far as Mos Eisley Spaceport - but he can't possibly leave 
            his Aunt and Uncle behind for some space adventure.
            ''',
        ),
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element='Crossing the First Threshold',
            description='''
            When Luke discovers that the stormtroopers searching for the droids 
            would track them to his farm, he rushes to warn his Aunt and Uncle, 
            only to discover them dead by the hands of the Empire. When Luke 
            returns to Kenobi, he pledges to go with him to Alderaan and learn 
            the ways of the Force like his father before him.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='Tests, Allies, Enemies',
            description='''
            After Luke, Kenobi, and the droids hire Han Solo and Chewbacca to 
            transport them onto Alderaan, Kenobi begins Luke's training in the 
            ways of the Force. Wielding his father's lightsaber, Kenobi 
            challenges Luke. At first, he can't do it. But then Kenobi tells 
            Luke to reach out and trust his feelings. Luke succeeds.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='Approach to the Inmost Cave',
            description='''
            The plan to defeat the Galactic Empire is to bring the Death Star 
            plans to Alderaan so that Princess Leia's father can take them to 
            the Rebellion. However, when they arrive within the system, the 
            planet is destroyed. They come across the Death Star and are pulled 
            in by a tractor beam, now trapped within the Galactic Empire.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element='Ordeal',
            description='''
            As Kenobi goes off to deactivate the tractor beam so they can escape, 
            Luke, Han, and Chewbacca discover that Princess Leia is being held on 
            the Death Star with them. They rescue her and escape to the Millennium 
            Falcon, hoping that Kenobi has successfully deactivated the tractor 
            beam. Kenobi later sacrifices himself as Luke watches Darth Vader 
            strike him down. Luke must now avenge his fallen mentor and carry on 
            his teachings.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element='Reward',
            description='''
            Luke has saved the princess and retrieved the Death Star plans. 
            They now have the knowledge to destroy the Galactic Empire's 
            greatest weapon once and for all.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='The Road Back',
            description='''
            Luke, Leia, Han, Chewbacca, and the droids are headed to the hidden 
            Rebellion base with the Death Star plans. They are suddenly pursued 
            by incoming TIE-Fighters, forcing Han and Luke to take action to 
            defend the ship and escape with their lives - and the plans. They 
            race to take the plans to the Rebellion and prepare for battle.
            ''',
        ),
        SceneBeat(
            location='On fighter ship X-Wing',
            plot_element='The Resurrection',
            description='''
            The Rebels - along with Luke as an X-Wing pilot - take on the Death 
            Star. The Rebellion and the Galactic Empire wage war in an epic space 
            battle. Luke is the only X-Wing pilot that was able to get within the 
            trenches of the Death Star. But Darth Vader and his wingmen are in hot 
            pursuit. Just as Darth Vader is about to destroy Luke, Han returns and 
            clears the way for Luke. Luke uses the Force to guide his aiming as he 
            fires upon the sole weak point of the deadly Death Star, destroying it 
            for good.
            ''',
        ),
        SceneBeat(
            location='At the Rebellion base',
            plot_element='The Return',
            description='''
            Luke and Han return to the Rebellion base, triumphant, as they receive 
            medals for the heroic journey. There is peace throughout the galaxy - at 
            least for now.
            ''',
        ),
    ],
) 
    

Dramatron's first generation step is creating a title and character list based on a user defined logline. Prompt instructions are all from the paper.

In [ ]:
from promptx import load

load(
    default_llm='chatgpt',
    openai_api_key='YOUR_API_KEY',
    openai_org_id='YOUR_ORG_ID',
)

Alternatively, you can set env variables which will be used to automatically load `promptx` when it's imported. 

```bash
export PXX_DEFAULT_LLM=chatgpt
export PXX_OPENAI_API_KEY=...
export PXX_OPENAI_ORG_ID=...
```

In [3]:
from promptx import prompt

def write_title(story: Story) -> str:
    return prompt(
        'Suggest a alternative, original and descriptive title for a known story.',
        story.logline,
        examples = [
            (
                star_wars.logline,
                "The Death Star's Menace"
            ),
            (
                "Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.",
                "The Day The Earth Was Saved By Outer Space."
            )
        ]
    )

story = Story(
    '''
    A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.
    '''
)

title = write_title(story)
story.title = title
story

2023-11-01 00:57:38.820 | INFO     | promptx:load:121 - loading local app from /home/rjl/promptx/examples/dramatron
2023-11-01 00:57:38.822 | INFO     | promptx:load:124 - loaded environment variables from /home/rjl/promptx/examples/dramatron/.env
2023-11-01 00:57:38.824 | INFO     | promptx:load:125 - API KEY wMeGC



Story(
    logline='\n    A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.\n    ',
    title='The Matrix Unveiled',
    outline=None,
    characters=None,
    locations=None
)

Next we create a list of character objects based on the logline by setting `output=[Character]`. This instructs the model to generate a JSON list of characters and `promptx` then parses the response into a list of instantiated `Character` objects.

In [4]:
import json

def create_characters(story: Story, n=5) -> List[Character]:
    return prompt(
        f'Create {n} characters for a story.',
        input=story.logline,
        output=[Character],
        examples=[
            (
                star_wars.logline,
                star_wars.characters,
            ),
        ],
    ).objects

characters = create_characters(story)
story.characters = characters
story


Story(
    logline='\n    A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.\n    ',
    title='The Matrix Unveiled',
    outline=None,
    characters=[
        Character(
            id='74ffea40-7bd1-4d8e-8c0f-10df8ce0e6e2',
            type='character',
            name='Neo',
            description='\n            Neo is the protagonist. He is a computer hacker who learns \n            about the true nature of reality and joins the rebels in the \n            war against the controllers.\n            '
        ),
        Character(
            id='a07f4e08-0c03-4175-9c0f-dde9c34f24db',
            type='character',
            name='Trinity',
            description="\n            Trinity is a skilled hacker and member of the rebel group. \n            She becomes Neo's love interest and fights alongside him in \n            the war.\n            "
        ),
        Character(
            id='d363d

With the logline, title, and characters generated, we can now move on to created the outline, which consists of a list of scene 'beats'.

In [5]:
def write_beats(story: Story, n=10) -> List[SceneBeat]:
    return prompt(
        f'''
        Write a sequence of {n} scene beats for a story a hero's journey structure.
        ''',
        input=dict(logline=story.logline, characters=story.characters),
        output=[SceneBeat],
        examples=[
            (
                dict(
                    logline=star_wars.logline,
                    characters=star_wars.characters,
                ),
                star_wars.outline,
            ),
        ],
    ).objects

beats = write_beats(story)
story.outline = beats
story


Story(
    logline='\n    A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.\n    ',
    title='The Matrix Unveiled',
    outline=[
        Scenebeat(
            id='b671a28b-44f4-4810-9e9c-bef923637bc6',
            type='scenebeat',
            location="Neo's apartment",
            plot_element='The Ordinary World',
            description='Neo is living a mundane life as a computer hacker, unaware of the true nature of reality.'
        ),
        Scenebeat(
            id='a77ee0a1-3b71-43ce-9c70-4e7cebf9f6ac',
            type='scenebeat',
            location='Mysterious nightclub',
            plot_element='Call to Adventure',
            description='\n            Neo is introduced to Trinity, who leads him to Morpheus. Morpheus tells Neo that the world he knows is a simulation called the matrix, and offers him the choice to learn the truth about reality and join the rebels in their fight aga

Next we extract the scene 'beats' from the plot outline generated in the previous step. These are the main events that happen in the story.

Each scene beat has a location name so we can use this to extract location objects. These are stored like characters. For some reason the examples in the paper don't use locations from Star Wars. They also use the story logline and just the name of the location from the scene beat instead of the description of the scene. It's unclear why these decisions were made, but for consistency we'll do the same.

In [6]:
def extract_locations(story: Story) -> List[Location]:
    locations = []
    for beat in story.outline:
        response = prompt(
            '''
            Generate a location based on the story logline and location name. 
            If the location is already known, return None
            ''',
            input=dict(logline=story.logline, name=beat.location, known_locations=[l.name for l in locations]),
            output=Location,
            examples=[
                (
                    dict(
                        logline="Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.",
                        name="The Adoption Center",
                        known_locations=["Harukiya"],
                    ),
                    Location(
                        name="The Adoption Center",
                        description='''
                        The Adoption Center is a sad place, especially for an unadopted 
                        pet. It is full of walls and walls of cages and cages. Inside of 
                        each is an abandoned animal, longing for a home. The lighting is 
                        dim, gray, buzzing fluorescent.
                        ''',
                    )
                ),
                (
                    dict(
                        logline="Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.",
                        name="The Adoption Center",
                        known_locations=["Harukiya", "The Adoption Center"],
                    ),
                    None,
                ),
                (
                    dict(
                        logline="James finds a well in his backyard that is haunted by the ghost of Sam.",
                        name="The Well",
                    ),
                    Location(
                        name="The Well",
                        description='''
                        The well is buried under grass and hedges. It is at least 
                        twenty feet deep, if not more and it is masoned with stones. 
                        It is 150 years old at least. It stinks of stale, standing 
                        water, and has vines growing up the sides. It is narrow enough 
                        to not be able to fit down if you are a grown adult human.
                        ''',
                    )
                ),
                (
                    dict(
                        logline="Mr. Dorbenson finds a book at a garage sale that tells the story of his own life. And it ends in a murder! ",
                        name="The Garage Sale",
                    ),
                    Location(
                        name="The Garage Sale",
                        description='''
                        It is a garage packed with dusty household goods and antiques. 
                        There is a box at the back that says FREE and is full of paper 
                        back books.
                        ''',
                    )
                ),
            ],
        )
        if response is not None:
            locations.append(response)
    return locations
    

locations = extract_locations(story)
story.locations = locations
story


Story(
    logline='\n    A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.\n    ',
    title='The Matrix Unveiled',
    outline=[
        Scenebeat(
            id='b671a28b-44f4-4810-9e9c-bef923637bc6',
            type='scenebeat',
            location="Neo's apartment",
            plot_element='The Ordinary World',
            description='Neo is living a mundane life as a computer hacker, unaware of the true nature of reality.'
        ),
        Scenebeat(
            id='a77ee0a1-3b71-43ce-9c70-4e7cebf9f6ac',
            type='scenebeat',
            location='Mysterious nightclub',
            plot_element='Call to Adventure',
            description='\n            Neo is introduced to Trinity, who leads him to Morpheus. Morpheus tells Neo that the world he knows is a simulation called the matrix, and offers him the choice to learn the truth about reality and join the rebels in their fight aga

Finally, we can write a script for each scene beat using the characters and locations generated in the previous steps.

In [7]:
def write_scene(story: Story, beat: SceneBeat) -> str:
    try:
        location = next(filter(lambda x: x.name.lower() == beat.location.lower(), story.locations))
    except StopIteration:
        print(f'No location found for {beat.location}')
        return None
    
    return prompt(
        f'''
        Write a scene for a story based on the scene beat and location.
        ''',
        input=dict(
            plot_element=beat.plot_element,
            beat_description=beat.description,
            location=location.name,
            characters=story.characters,
            logline=story.logline,
            title=story.title,
        ),
    )


def write_script(story: Story) -> list[str]:
    return [write_scene(story, beat) for beat in story.outline]

In [8]:
script = write_script(story)
script


[
    "INT. NEO'S APARTMENT - NIGHT\n\nThe room is dimly lit, with stacks of computer equipment scattered around. Posters of vintage video games adorn the walls. Neo, a disheveled and tired-looking man in his late twenties, sits hunched over a desk, engrossed in his hacking activities.\n\nNARRATOR (V.O.)\nNeo is living a mundane life as a computer hacker, unaware of the true nature of reality.\n\nNeo's fingers dance across the keyboard as lines of code scroll rapidly on the screen. Sweat beads on his forehead, showing his intense concentration.\n\nSuddenly, the sound of footsteps can be heard approaching the door. Neo quickly minimizes his hacking tools and pretends to browse the internet innocently.\n\nThe door bursts open, and TRINITY, a confident and enigmatic woman in her thirties, enters the apartment. She holds a small device in her hand, scanning the room with a sharp gaze.\n\nTRINITY\n(smirking)\nCaught you again, Neo.\n\nNeo looks up, startled, and a mixture of relief and int

Let's put all of that together in a `create_story` function that takes a single logline as input and returns a fully generated story object.

In [9]:
def create_story(logline: str) -> Story:
    story = Story(logline=logline)
    story.title = write_title(story)
    story.characters = create_characters(story)
    story.outline = write_beats(story)
    story.locations = extract_locations(story)
    return story

In [10]:
uncut_gems = create_story(
    '''
    Howard Ratner, a charismatic jeweler in New York City, makes a high-stakes bet that could lead to the windfall of a lifetime.
    '''
)

uncut_gems_script = write_script(uncut_gems)
uncut_gems_script

In [12]:
uncut_gems_script


[
    "INT. HOWARD RATNER'S JEWELRY STORE - DAY\n\nThe camera pans over an array of dazzling diamonds and sparkling gold jewelry, capturing the glimmer and glamour of the jewelry store. Customers browse the displays, admiring the exquisite craftsmanship.\n\nHOWARD RATNER, a charismatic jeweler, stands behind the counter, wearing a flashy suit and a mischievous grin. He exudes confidence and charm as he interacts with his customers. \n\nHoward's attention is caught by a GROUP OF MEN huddled near the entrance, speaking in hushed tones. He senses something exciting is about to happen.\n\nHOWARD RATNER\n(leaning in, intrigued)\nWhat's going on, fellas?\n\nThe men turn to face Howard, their expressions a mix of excitement and caution.\n\nMAN 1\n(whispering)\nHoward, there's a rumor going around town. A rare black opal has been smuggled into New York City.\n\nHoward's eyes widen with interest. Black opals are highly sought after, known for their exceptional beauty and incredible value.\n\nH